## Futures Roll

In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
from scipy.stats import norm
import os, sys
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import zipfile
import urllib.request
from PIL import Image
import importlib
from threading import Thread
import time

# Make important folders
TEMP_FOLDER = './temp_folder'
try:
    os.mkdir(TEMP_FOLDER)
except:
    pass
SAVE_CSV_FOLDER = f'{TEMP_FOLDER}/ib'
try:
    os.mkdir(SAVE_CSV_FOLDER)
except:
    pass

def to_int(s):
    try:
        return int(float(str(s)))
    except:
        print(f'to_int exception on value:{s}')
        return None

pd.set_option('display.max_colwidth',1000)
 

In [2]:
nymex_sd_dict = {s:'NYMEX' for s in ['CL','NG','HO','RB','PL','PA']}
mcs_lists = {c:'FGHJKMNQUVXZ' for c in nymex_sd_dict.keys()}
secdef_dict = nymex_sd_dict.copy()

ecbot_hknuz_sd_dict = {s:'ECBOT' for s in ['ZW','ZC','ZS']}
mcs_lists.update({c:'HKNUZ' for c in ecbot_hknuz_sd_dict.keys()})
secdef_dict.update(ecbot_hknuz_sd_dict)

ecbot_hmuz_sd_dict = {s:'ECBOT' for s in ['ZN','ZB']}
mcs_lists.update({c:'HMUZ' for c in ecbot_hmuz_sd_dict.keys()})
secdef_dict.update(ecbot_hmuz_sd_dict)

ecbot_fhknqux_sd_dict = {s:'ECBOT' for s in ['ZS']}
mcs_lists.update({c:'FHKNQUX' for c in ecbot_fhknqux_sd_dict.keys()})
secdef_dict.update(ecbot_fhknqux_sd_dict)

cme_hmuz_sd_dict = {s:'GLOBEX' for s in ['ES','GE']}
mcs_lists.update({c:'HMUZ' for c in cme_hmuz_sd_dict.keys()})
secdef_dict.update(cme_hmuz_sd_dict)

cash_hmuz_sd_dict = {s:'IDEALPRO' for s in ['EUR','GB','SF']}
mcs_lists.update({c:'HMUZ' for c in cash_hmuz_sd_dict.keys()})
secdef_dict.update(cash_hmuz_sd_dict)


month_codes = ','.join('FGHJKMNQUVXZ').split(',')
monthcode_to_monthnum = {month_codes[i]:i for i in range(len(month_codes))}
cash_list = list(cash_hmuz_sd_dict.keys())
cash_list

['EUR', 'GB', 'SF']

In [3]:
def fetch_ib_history(contract,days_back=60,time_period=86400000):
    url = f"http://127.0.0.1:8899/ibhistory?{time_period}%20{days_back}%200%205%20{contract}"
    df = pd.read_csv(url)
    return df
    

In [4]:
def fetch_ib_fut_history(symbol,currency='USD',days_back=60):
    if len(symbol)>3:
        commod = symbol[:-3]
    else:
        commod = symbol
    exch = secdef_dict[commod]
    if commod in cash_list:
        contract = ".".join([commod,'CASH',exch])
    else:
        y = str(2000 + int(symbol[-2:]))
        m = '%02d' %(monthcode_to_monthnum[symbol[-3:-2]] + 1)
        contract = ".".join([commod,'FUT',exch,currency,y])+m
    print(contract)
    url = f"http://127.0.0.1:8899/ibhistory?86400000%20{days_back}%200%205%20{contract}"
    df = pd.read_csv(url)
    df['trade_date'] = df.date.apply(lambda d: int(str(d)[0:4] + str(d)[5:7] + str(d)[8:10]))
    df.date = df.date.apply(lambda d: str(d)[0:4] + "-" + str(d)[4:6] + "-" + str(d)[6:8])
    df = df.sort_values('date')
    df.index = list(range(len(df)))
    # make adj close the close
    df['nav'] = df['adjusted']
    df['symbol'] = symbol
    df['shares'] = 0
    df = df[['symbol','date','nav','shares','trade_date']]
    return df

In [5]:
def get_spot_code(commod):
    mcs = mcs_lists[commod]
    n = datetime.datetime.now()
    endy = n.year - 2000
    begy = endy - 2
    m = n.month 
    code_this_month = month_codes[m-1]
    spot_code = None
    for c in mcs:
        if c>=code_this_month:
            spot_code=c
            break
    if spot_code is None:
        spot_code = mcs[0]
        endy +=1
    return spot_code

In [6]:
def get_series(commod):
    mcs = mcs_lists[commod]
    n = datetime.datetime.now()
    endy = n.year - 2000
    begy = endy - 2
    spot_code = get_spot_code(commod)
    last_contract = spot_code + '%02d' %(endy)
#     print(last_contract,mcs,code_this_month)
    years = np.linspace(begy,endy,endy-begy+1,dtype=int)
    for y in years:
        for mc in mcs:
            symbol = '%s%s%02d' %(commod,mc,y)
            save_file = f'{SAVE_CSV_FOLDER}/{symbol}.csv'
            if os.path.isfile(save_file):
                continue
            print(f'fetch symbol {symbol}')
            df = fetch_ib_fut_history(symbol)
            df.to_csv(save_file)
            if symbol[-3:]==last_contract:
                break


### Test  fetch_ib_history

In [ ]:
df = fetch_ib_history('FXE.STK.SMART')

### Test fetch_ib_fut_history

In [ ]:
s = 'CL'
c = s+get_spot_code(s)+'19'
fetch_ib_fut_history(symbol=c)

## get all series

In [8]:
def get_series_async(commod):
    print(commod)
    try:
        get_series(commod)
    except Exception as e:
        print(str(e))
        

contract_dict = {}
for commod in secdef_dict.keys():
    if commod not in contract_dict.keys():
#         t = Thread(target=get_series_async,args=(commod,))
#         t.start()
        print(commod)
        try:
            get_series(commod)
        except Exception as e:
            print(str(e))

CL
fetch symbol CLF17
CL.FUT.NYMEX.USD.201701
fetch symbol CLG17
CL.FUT.NYMEX.USD.201702
fetch symbol CLH17
CL.FUT.NYMEX.USD.201703
fetch symbol CLX17
CL.FUT.NYMEX.USD.201711
fetch symbol CLZ17
CL.FUT.NYMEX.USD.201712
fetch symbol CLF18
CL.FUT.NYMEX.USD.201801
fetch symbol CLG18
CL.FUT.NYMEX.USD.201802
fetch symbol CLH18
CL.FUT.NYMEX.USD.201803
fetch symbol CLJ18
CL.FUT.NYMEX.USD.201804
fetch symbol CLK18
CL.FUT.NYMEX.USD.201805
fetch symbol CLM18
CL.FUT.NYMEX.USD.201806
fetch symbol CLN18
CL.FUT.NYMEX.USD.201807
fetch symbol CLQ18
CL.FUT.NYMEX.USD.201808
fetch symbol CLU18
CL.FUT.NYMEX.USD.201809
fetch symbol CLV18
CL.FUT.NYMEX.USD.201810
fetch symbol CLX18
CL.FUT.NYMEX.USD.201811
fetch symbol CLZ18
CL.FUT.NYMEX.USD.201812
fetch symbol CLF19
CL.FUT.NYMEX.USD.201901
fetch symbol CLG19
CL.FUT.NYMEX.USD.201902
fetch symbol CLH19
CL.FUT.NYMEX.USD.201903
NG
fetch symbol NGF17
NG.FUT.NYMEX.USD.201701
fetch symbol NGG17
NG.FUT.NYMEX.USD.201702
fetch symbol NGH17
NG.FUT.NYMEX.USD.201703
fetch

In [28]:
df.adjusted.pct_change().iloc[1:].std() * 256**.5

0.0550206442808462